## Notebook config

In [1]:
CPU_THREADS = 1          ## Number of CPU to use
MEM_LIMIT_GB = 2         ## Default value for running the notebook in binder
KMERS = '27,31'          ## List og k-mers to use for assembly
USE_RAW_FASTQ = True     ## Download raw fastq instead of SRA format, faster on binder
SUBSAMPLE_READ = 1000000 ## subsample reads to 1M to run in binder, set it to zero to disable

In [2]:
list_of_samples = \
  [{'sample_name':'SRR10971381',
    'fastq_files' : [
      '/tmp/SRR10971381_1.fastq.gz',
      '/tmp/SRR10971381_2.fastq.gz']}
  ]

In [5]:
import os, requests

### Step 1: Fetch fastq read from SRA

In [3]:
if USE_RAW_FASTQ:
  ## Download raw fastq from SRA
  !wget -O /tmp/SRR10971381_1.fastq.gz https://sra-pub-src-1.s3.amazonaws.com/SRR10971381/WH_R1.fastq.gz.1
  !wget -O /tmp/SRR10971381_2.fastq.gz https://sra-pub-src-1.s3.amazonaws.com/SRR10971381/WH_R2.fastq.gz.1
else:
  ## Download reads in SRA format
  !wget -O /tmp/SRR10971381 https://sra-download.ncbi.nlm.nih.gov/traces/sra46/SRR/010714/SRR10971381
  ## Convert SRA format data to fastq format
  !fastq-dump --split-files --gzip -outdir /tmp /tmp/SRR10971381

--2020-04-08 21:06:38--  https://sra-pub-src-1.s3.amazonaws.com/SRR10971381/WH_R1.fastq.gz.1
Resolving sra-pub-src-1.s3.amazonaws.com (sra-pub-src-1.s3.amazonaws.com)... 52.216.107.236
Connecting to sra-pub-src-1.s3.amazonaws.com (sra-pub-src-1.s3.amazonaws.com)|52.216.107.236|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2739477612 (2.6G) [application/x-troff-man]
Saving to: '/tmp/SRR10971381_1.fastq.gz'

/tmp/SRR10971381_1. 100%[===================>]   2.55G  32.3MB/s    in 89s     

2020-04-08 21:08:07 (29.4 MB/s) - '/tmp/SRR10971381_1.fastq.gz' saved [2739477612/2739477612]

--2020-04-08 21:08:09--  https://sra-pub-src-1.s3.amazonaws.com/SRR10971381/WH_R2.fastq.gz.1
Resolving sra-pub-src-1.s3.amazonaws.com (sra-pub-src-1.s3.amazonaws.com)... 52.216.102.43
Connecting to sra-pub-src-1.s3.amazonaws.com (sra-pub-src-1.s3.amazonaws.com)|52.216.102.43|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2838458153 (2.6G) [application/x-tro

### Step 2: Fetch Coronavirus genome and unique kmers

In [4]:
## fetch coronavirus genomes

!wget -O /tmp/SARS2_153_complete_genomes_20200329.fasta https://storage.googleapis.com/sars-cov-2/SARS2_153_complete_genomes_20200329.fasta

--2020-04-08 21:14:05--  https://storage.googleapis.com/sars-cov-2/SARS2_153_complete_genomes_20200329.fasta
Resolving storage.googleapis.com (storage.googleapis.com)... 108.177.120.128, 2607:f8b0:4001:c05::80
Connecting to storage.googleapis.com (storage.googleapis.com)|108.177.120.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4662317 (4.4M) [application/octet-stream]
Saving to: '/tmp/SARS2_153_complete_genomes_20200329.fasta'

/tmp/SARS2_153_comp 100%[===================>]   4.45M  --.-KB/s    in 0.07s   

2020-04-08 21:14:05 (60.9 MB/s) - '/tmp/SARS2_153_complete_genomes_20200329.fasta' saved [4662317/4662317]



In [5]:
## fetch unique kmers

!wget -O /tmp/SARS-CoV-2.kmer.fa http://opengene.org/fastv/data/SARS-CoV-2.kmer.fa

--2020-04-08 21:14:09--  http://opengene.org/fastv/data/SARS-CoV-2.kmer.fa
Resolving opengene.org (opengene.org)... 47.90.42.109
Connecting to opengene.org (opengene.org)|47.90.42.109|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 7632 (7.5K) [application/octet-stream]
Saving to: '/tmp/SARS-CoV-2.kmer.fa'

/tmp/SARS-CoV-2.kme 100%[===================>]   7.45K  --.-KB/s    in 0s      

2020-04-08 21:14:09 (1.07 GB/s) - '/tmp/SARS-CoV-2.kmer.fa' saved [7632/7632]



## Sub-sample reads for binder run

In [3]:
## this step may take some time to run
for entry in list_of_samples:
  if SUBSAMPLE_READ > 0:
    sample_name = entry.get('sample_name')
    fastq_files = entry.get('fastq_files')
    R1_fastq = fastq_files[0]
    R2_fastq = fastq_files[1]
    R1_sub_fastq = '/tmp/{0}_sub_1.fastq'.format(sample_name)
    R2_sub_fastq = '/tmp/{0}_sub_2.fastq'.format(sample_name)
    ## running seqtk to subsample files 
    print('subsampling reads for sample {0} R1'.format(sample_name))
    !seqtk sample -2 -s100 $R1_fastq $SUBSAMPLE_READ  > $R1_sub_fastq
    print('subsampling reads for sample {0} R2'.format(sample_name))
    !seqtk sample -2 -s100 $R2_fastq $SUBSAMPLE_READ > $R2_sub_fastq
    entry.update({'subsample_fastq_files':[R1_sub_fastq,R2_sub_fastq]})

subsampling reads for sample SRR10971381 R1
subsampling reads for sample SRR10971381 R2


### Step 3: Check for viral contamination using Fastv

In [9]:
for entry in list_of_samples:
  sample_name = entry.get('sample_name')
  if SUBSAMPLE_READ > 0:
    fastq_files = entry.get('subsample_fastq_files')
  else:
    fastq_files = entry.get('fastq_files')

  R1_fastq = fastq_files[0]
  R2_fastq = fastq_files[1]
    
  ## run fastv
  print('running fastv for sample {0}'.format(sample_name))
  fastv_html_output = '{0}.fastv.html'.format(sample_name)
  fastv_json_output = '{0}.fastv.json'.format(sample_name)
  fastv_log_output = '{0}.fastv.log'.format(sample_name)
  !~/bin/fastv \
    -i $R1_fastq \
    -I $R2_fastq \
    -k /tmp/SARS-CoV-2.kmer.fa \
    -g /tmp/SARS2_153_complete_genomes_20200329.fasta \
    -h $fastv_html_output \
    -j $fastv_json_output \
    --thread $CPU_THREADS 2> $fastv_log_output

running fastv for sample SRR10971381


## Step 4: De-novo genome assembly

In [9]:
MEM_LIMIT_BYTES = MEM_LIMIT_GB * 1000000000

for entry in list_of_samples:
  sample_name = entry.get('sample_name')
  if SUBSAMPLE_READ > 0:
    fastq_files = entry.get('subsample_fastq_files')
  else:
    fastq_files = entry.get('fastq_files')

  R1_fastq = fastq_files[0]
  R2_fastq = fastq_files[1]

  output_dir = 'megahit_assembly_{0}'.format(sample_name)
  ## run megahit assembly
  print('running de-novo assembly using megahit for sample {0}'.format(sample_name))
  !megahit \
    -1 $R1_fastq \
    -2 $R2_fastq \
    -o $output_dir \
    --k-list $KMERS \
    --num-cpu-threads $CPU_THREADS \
    --memory $MEM_LIMIT_BYTES \
    --tmp-dir /tmp
  ### Visualize assembly
  entry.update({'assembly_output':output_dir})
  print('converting assembly to fastg file for visualization')
  max_kmer = KMERS.split(',')[-1]
  fastg_input = \
    os.path.join(
      output_dir,
      'intermediate_contigs',
      'k{0}.contigs.fa'.format(max_kmer))
  fastg_output = '{0}_k{1}.fastg'.format(sample_name,max_kmer)
  !megahit_toolkit contig2fastg $max_kmer $fastg_input > $fastg_output

running de-novo assembly using megahit for sample SRR10971381
2020-04-08 22:57:10 - MEGAHIT v1.2.9
2020-04-08 22:57:10 - Using megahit_core with POPCNT and BMI2 support
2020-04-08 22:57:10 - Convert reads to binary library
2020-04-08 22:57:13 - b'INFO  sequence/io/sequence_lib.cpp  :   77 - Lib 0 (/tmp/SRR10971381_sub_1.fastq,/tmp/SRR10971381_sub_2.fastq): pe, 2000000 reads, 151 max length'
2020-04-08 22:57:13 - b'INFO  utils/utils.h                 :  152 - Real: 2.5765\tuser: 1.9858\tsys: 0.4955\tmaxrss: 162244'
2020-04-08 22:57:13 - Start assembly. Number of CPU threads 1 
2020-04-08 22:57:13 - k list: 27,31 
2020-04-08 22:57:13 - Memory used: 2000000000
2020-04-08 22:57:13 - Extract solid (k+1)-mers for k = 27 
2020-04-08 22:58:58 - Build graph for k = 27 
2020-04-08 23:00:22 - Assemble contigs from SdBG for k = 27
2020-04-08 23:05:33 - Local assembly for k = 27
2020-04-08 23:05:51 - Extract iterative edges from k = 27 to 31 
2020-04-08 23:06:15 - Build graph for k = 31 
2020-04-08

In [10]:
def fetch_genome_fasta_from_ncbi(refseq_id,output_path='.'):
  '''
  A function for fetching the genome fasta sequences from NCBI
  
  :param refseq_id: NCBI genome id
  :param output_path: Path to dump genome files, default '.'
  '''
  try:
    url = \
      'https://eutils.ncbi.nlm.nih.gov/entrez/eutils/efetch.fcgi?db=nucleotide&id={0}&rettype=fasta'.\
        format(refseq_id)
    r = requests.get(url)
    fasta_data = r.content.decode('utf-8')
    output_file = \
      os.path.join(
        os.path.abspath(output_path),
        '{0}.fa'.format(refseq_id))
    with open(output_file,'w') as fp:
      fp.write(fasta_data)
    print('Downloaded genome seq for {0}'.format(refseq_id))
  except Exception as e:
    print('Failed to download data for {0} from NCBI, error: {1}'.format(refseq_id,e))

In [11]:
fetch_genome_fasta_from_ncbi(refseq_id='NC_045512.2')

Downloaded genome seq for NC_045512.2


## Align contigs using Mauve

In [13]:
for entry in list_of_samples:
  sample_name = entry.get('sample_name')
  assembly_output = entry.get('assembly_output')
  ## run mauve alignment
  mauve_input = os.path.join(assembly_output,'final.contigs.fa')
  mauve_output = 'mauve_contig_order_{0}'.format(sample_name)
  mauve_log = '{0}_mauve.log'.format(sample_name)
  !java -Xmx2000m -Djava.awt.headless=true -cp \
     ~/bin/mauve_snapshot_2015-02-13/Mauve.jar \
     org.gel.mauve.contigs.ContigOrderer \
     -output $mauve_output \
     -ref NC_045512.2.fa \
     -draft $mauve_input > $mauve_log

Apr 08, 2020 11:14:53 PM java.util.prefs.FileSystemPreferences$1 run
INFO: Created user preferences directory.


In [15]:
!head mauve_contig_order_SRR10971381/alignment2/final.contigs.fa_contigs.tab

Contigs to reverse
type	label	contig	strand	left_end	right_end
contig	k31_39596	chromosome	complement	520307	523223
contig	k31_61516	chromosome	complement	808899	809946
contig	k31_94094	chromosome	complement	1170224	1174677
contig	k31_116196	chromosome	complement	1397534	1406147
contig	k31_155347	chromosome	complement	1789324	1789589


Ordered Contigs
